In [ ]:
from agentjo import *

def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
from agentjo.agent import Agent,BaseAgent
from agentjo.base import strict_json
import re
from termcolor import colored

class CustomizationWrapper(Agent):
    def __init__(self,agent: Agent, custom_functions: dict = None,verbose: bool = True):
        # Initialize the parent Agent
        super().__init__(**agent.__dict__)  # Inherit all of the attributes of the passed agent
        # Initiatlize the functions
        self.assign_functions(list(agent.function_map.values()))
        self.custom_functions = custom_functions
        self.assign_custom_functions()

    def assign_custom_functions(self):
        #ensure all keys are in custom_functions and other keys dont exist
        expected_keys = ['on_run','before_subtask','subtask_completed','on_reply_user']
        for key in self.custom_functions.keys():
            if key not in expected_keys:
                raise ValueError(f"Custom function key {key} not in allowed keys {expected_keys}")
        for key in expected_keys:
            if key not in self.custom_functions.keys():
                self.custom_functions[key] = []

        for phase, function_list in self.custom_functions.items():
            new_function_list = []
            #replace function_list with the list of functions you want to use
            for function in function_list:
                if isinstance(function, BaseAgent):
                    function = function.to_function(self)
            
                # do automatic conversion of function to Function class (this is in base.py)
                if not isinstance(function, Function):
                    function = Function(external_fn = function)
                
                new_function_list.append(function)

            self.custom_functions[phase] = new_function_list


    def use_custom_functions(self,phase):

        #use the custom functions in the phase
        if phase in self.custom_functions:
            functions = self.custom_functions[phase]
            if len(functions) >0:
                if self.verbose: 
                        print(f'Calling functions for phase: {phase}')

                for cur_function in functions:
                    function_name = cur_function.fn_name
                    input_format = {}
                    fn_description = cur_function.fn_description
                    matches = re.findall(r'<(.*?)>', fn_description)
                    
                    # do up an output format dictionary to use to get LLM to output exactly based on keys and types needed
                    for match in matches:
                        if ':' in match:
                            first_part, second_part = match.split(':', 1)
                            input_format[first_part] = f'A suitable value, type: {second_part}'
                        else:
                            input_format[match] = 'A suitable value'
                            
                    # if there is no input, then do not need LLM to extract out function's input
                    if input_format == {}:
                        function_params = {}
                            
                    else:
                        background_info = f"Assigned Task:```\n{self.task}\n```\nSubtasks Completed: ```{self.subtasks_completed}```"
                        # Add in memory to the Agent
                        rag_info = ''
                        for name in self.memory_bank.keys():
                            # Function RAG is done separately in self.query()
                            if name == 'Function': continue
                            rag_info += f'Knowledge Reference for {name}: ```{self.memory_bank[name].retrieve(task)}```\n'    

                        function_params = self.query(query = f'''{background_info}{rag_info}\n\n```\nEquipped Function Details: ```{str(cur_function)}```\nOutput suitable values for Inputs to Equipped Function to fulfil Current Subtask\nInput fields are: {list(input_format.keys())}''',
                                    output_format = input_format,
                                    provide_function_list = False)
                    

                    # Execute the function for next step
                    res = cur_function(shared_variables = self.shared_variables, **function_params)

                    # Stateful collection of function outcomes
                    if res == '':
                        res = {'Status': 'Completed'}
                        
                    formatted_subtask = function_name + '(' + ", ".join(f'{key}="{value}"' if isinstance(value, str) else f"{key}={value}" for key, value in function_params.items()) + ')'
                    self.add_subtask_result(formatted_subtask, res)

    def custom_run(self, task: str = '', overall_task: str = '', num_subtasks: int = 0) -> list:
        ''' Attempts to do the task using LLM and available functions
        Loops through and performs either a function call or LLM call up to num_subtasks number of times
        If overall_task is filled, then we store it to pass to the inner agents for more context '''
            
        # Assign the task
        if task != '':
            self.task_completed = False
            # If meta agent's task is here as well, assign it too
            if overall_task != '':
                self.assign_task(task, overall_task)
            else:
                self.assign_task(task)
            
        # check if we need to override num_steps
        if num_subtasks == 0:
            num_subtasks = self.max_subtasks
        
        self.use_custom_functions('on_run')

        # if task completed, then exit
        if self.task_completed: 
            if self.verbose:
                print('Task already completed!\n')
                print('Subtasks completed:')
                for key, value in self.subtasks_completed.items():
                    print(f"Subtask: {key}\n{value}\n")
                    
        else:
            # otherwise do the task
            for i in range(num_subtasks):
                
                self.use_custom_functions('before_subtask')

                # Determine next subtask, or if task is complete. Always execute if it is the first subtask
                subtask, function_name, function_params = self.get_next_subtask()
                if function_name == 'end_task':
                    self.task_completed = True
                    if self.verbose:
                        print(colored(f"Subtask identified: End Task", "blue", attrs=['bold']))
                        print('Task completed successfully!\n')
                    break
                    
                if self.verbose: 
                    print(colored(f"Subtask identified: {subtask}", "blue", attrs=['bold']))

                # Execute the function for next step
                res = self.use_function(function_name, function_params, subtask)
                
                self.use_custom_functions('subtask_completed')

                # Summarise Subtasks Completed if necessary
                if len(self.subtasks_completed) > self.summarise_subtasks_count:
                    print('### Auto-summarising Subtasks Completed (Change frequency via `summarise_subtasks_count` variable) ###')
                    self.summarise_subtasks_completed(f'progress for {self.overall_task}')
                    print('### End of Auto-summary ###\n')
          
        return list(self.subtasks_completed.values())



In [19]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory'].append(item)
    return f'{item} successfully added to Inventory'
    
def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory']:
        shared_variables['Inventory'].remove(item)
        return f'{item} successfully removed from Inventory'
    else:
        return f'{item} not found in Inventory, unable to remove'
    
agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

output = agent.run('Add apples and oranges')

Observation: The inventory is currently empty, and the task is to add apples and oranges to it.
Thoughts: To complete the task, I need to add both apples and oranges to the inventory. Since the inventory is empty, I can start by adding apples first.
Subtask identified: Add apples to the inventory.
Calling function add_item_to_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully added to Inventory'}

Observation: Apples have been successfully added to the Inventory. The task requires adding both apples and oranges to the Inventory.
Thoughts: Since apples have already been added, the next step is to add oranges to the Inventory to complete the task.
Subtask identified: Add oranges to the Inventory.
Calling function add_item_to_inventory with parameters {'item': 'oranges'}
> {'output_1': 'oranges successfully added to Inventory'}

Observation: The items "apples" and "oranges" have already been successfully added to the Inventory as per the subtasks completed.
T

In [18]:
def reflect(shared_variables) -> str:
    '''Produces a thought about the task'''
    agent = shared_variables['agent']
    res = strict_json(system_prompt = f'''Reflect on the progress towards your main task {agent.overall_task} \n Subtasks Completed: ```{agent.subtasks_completed}```''',
                        user_prompt = '',
                        output_format = {'Reflection': 'Your reflections, type: str'},
                        llm = llm)
    print(res['Reflection'])
    return res['Reflection']

def plan(shared_variables) -> str:
    '''Produces a plan for the task'''
    agent = shared_variables['agent']
    agent_functions = [(key, value) for key, value in agent.function_map.items()]
    res = strict_json(system_prompt = f'''Your overall task is {agent.overall_task} \n Your available Equipped Functions are: {agent_functions} \n Create a reasonable plan to accomplish the task in the form of a list of functions you wish to execute by name. ''',
                        user_prompt = '',
                        output_format = {'Plan': 'List of function names to run in order, type: list'},
                        llm = llm)
    print(res['Plan'])
    return res['Plan']

my_agent = Agent('Inventory Manager', 
              'Adds and removes items in Inventory. Only able to remove items if present in Inventory',
              shared_variables = {'Inventory': []},
              global_context = 'Inventory: <Inventory>', # Add in Global Context here with shared_variables Inventory
              llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

my_agent = CustomizationWrapper(my_agent, custom_functions = {'on_run':[plan],'subtask_completed': [reflect]})

output = my_agent.custom_run('Add apples and oranges')

Calling functions for phase: on_run
['add_item_to_inventory', 'add_item_to_inventory', 'use_llm', 'end_task']
Observation: The plan indicates that the task is to add apples and oranges to the inventory. The inventory is currently empty, so both items need to be added.
Thoughts: Since the inventory is empty, I need to add apples first as per the plan. After adding apples, I will proceed to add oranges.
Subtask identified: Add apples to the inventory.
Calling function add_item_to_inventory with parameters {'item': 'apples'}
> {'output_1': 'apples successfully added to Inventory'}

Calling functions for phase: subtask_completed
The progress towards the main task of adding apples and oranges is partially complete. The subtask of adding apples to the inventory has been successfully completed. However, the task of adding oranges is still pending. Additionally, there is a step involving the use of a language model (use_llm) that has not been addressed yet. To move forward, I need to focus on 